#### About

KNN collaborative filtering approach to recommending beers based on a sparse dataset.

- num reviews: 4,837,392 (out of 5,487,730)
- num beers: 9,999 (out of 24,542)
- num users: 5,2707 (out of 101,574)

**item based approach** will allow the computations to be done offline and then served, as the items change less than users.
A user based approach would need to be updated and retrained too frequently.

**euclidean dist vs cosine similarity:**
- **cosine:**
- looks at the angle between two vectors without considering magnitude
- useful when comparing vectors of different length or dimensionality, helps balance the gap and prevent favoring samples based on number of dimensions rather than similarity between values.

**choosing a nearest neighbor algorithm:**
- **brute:** can be very slow for large datasets with high dimensionality
- **ball tree:** 
- recursively divide data into nodes defined by centroid C and radius r, which reduces the candidates to compare to a new data point. Builds a tree to filter new data points into the most similar node (brute force is then done within node).
- works well with sparse data that is highly intrinsic, but large portion of time is spent building the query tree relative to doing a single query. better when several queries are necessary (true for recommender!)
- leaf_size == node size, very high leaf size results in quick construction but closer query time to brute force. very low leaf size results in lots of time spent filtering through tree.

**choosing k:**
- **brute:** largely unnaffected by choice of k
- **ball tree:** can slow down with larger k partially due to internal queuing and increased difficulty pruning branches in query tree.

This code follows the blog about recommenders found [here](https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea)

In [45]:
import os
import time


import math
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

from fuzzywuzzy import fuzz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# has beer names
df_orig = pd.read_csv('./Beer_Data/reduced_data.csv')
# totally numeric, no beer names
df = pd.read_csv('./Beer_Data/reduced_data_X2.csv')
# beer names to ID's
beer_ids = pd.read_csv('./Beer_Data/beer_ids.csv')

In [3]:
df

,beer_id,user_score,user_id
0,18580,3.75,1
1,18570,4.25,1
2,18581,4.25,1
3,4200,4.25,1
4,1,4.50,1
...,...,...,...
4837387,3583,4.25,101906
4837388,14654,4.00,101906
4837389,1106,3.40,101906
4837390,11819,4.00,101906


In [6]:
# pivot ratings into movie features
df_beer_features = df.pivot(
    index='beer_id',
    columns='user_id',
    values='user_score'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_beer_features = csr_matrix(df_beer_features.values)

In [8]:
df_beer_features

user_id,1,2,3,4,5,6,7,8,9,11,...,101582,101595,101653,101664,101679,101717,101818,101857,101865,101906
beer_id,,,,,,,,,,,,,,,,,,,,,
1,4.50,5.00,4.25,4.0,3.5,3.91,3.53,3.85,4.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.25,3.59,4.50,0.0,3.0,4.00,0.00,0.00,4.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.25,4.50,4.25,0.0,4.0,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4.75,5.00,4.25,4.0,4.0,0.00,0.00,0.00,3.75,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.00,4.85,4.25,0.0,4.5,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24532,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24535,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24536,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# build predictor
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [43]:
# fit predictor with sparse matrix
model_knn.fit(df_beer_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [42]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, beer_id in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, beer_id, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

In [ ]:
def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    beer_id = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[beer_id], n_neighbors=n_recommendations+1)
    # get list of raw beer_id of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (beer_id, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[beer_id], dist))